In [40]:
import librosa
import glob
import os
import dtw
import matplotlib.pyplot as plt
import seaborn
seaborn.set(style='dark')
%matplotlib inline

import msaf # develop branch
from msaf import jams2
import numpy as np
import scipy
import mir_eval
import scipy.spatial.distance
from joblib import Parallel, delayed
import cPickle

In [60]:
ds_path = "/Users/uri/datasets/Segments/"
ds_path = "/home/uri/datasets/Segments/"

In [52]:
%run 'Cross-repetition DTW.ipynb'

IndexError: list index out of range

In [53]:
%run 'Repetition detection threshold.ipynb'

## Parameters to Explore ##

Datasets (levels):

* SALAMI (large_scale, small_scale)
* Isophonics (function)

Distances for the DTW scores:

* L1
* L2
* correlation

Normalization of the DTW scores to compute the threshold and the F-measures:

* None
* Minimum
* Maximum
* Harmonic Mean

In [58]:
datasets = ["SALAMI", "Isophonics"]
dist_dict = {
    "L1": lambda x, y: norm(x - y, ord=1), 
    "L2": scipy.spatial.distance.euclidean,
    "correlation": scipy.spatial.distance.correlation
}
dataset_levels = {
    "Isophonics": ["function"],
    "SALAMI": ["large_scale", "small_scale"]
}
norms = ["none", "min", "max", "hmean"]

In [59]:
def compute_score(file_struct, level, dist_key):
    try:
        ref_inter, ref_labels = jams2.converters.load_jams_range(
            file_struct.ref_file, "sections", annotator=0, context=level)
        D, P = make_cost_matrix(file_struct.audio_file, ref_inter, ref_labels, dist=dist_dict[dist_key])
        thresholds = {}
        fmeasures = {}
        for norm in norms:
            thresholds[norm], fmeasures[norm] = plot_curve(file_name=file_struct.audio_file, 
                                                           intervals=ref_inter, labels=ref_labels,
                                                           scores=D, norm=norm)
    except:
        print "warning: no annotations for file", file_struct.audio_file
        ref_inter = None
        ref_labels = None
        scores = None
        paths = None
        thresholds = None
        fmeasures = None
    ret = {
        "intervals": ref_inter,
        "labels": ref_labels,
        "scores": D,
        "paths": P,
        "thresholds": thresholds,
        "fmeasures": fmeasures,
        "file_name": os.path.basename(file_struct.audio_file)
    }
    return ret

In [ ]:
n_jobs = 8

for dataset in datasets:
    files = msaf.io.get_dataset_files(ds_path, ds_name=dataset)
    for level in dataset_levels[dataset]:
        for dist_key in dist_dict.keys():
            print "Computing: ", dataset, level, dist_key
            scores = Parallel(n_jobs=n_jobs)(delayed(compute_score)(
                file_struct, level, dist_key)
                for file_struct in files[:])
            result = {
                "level": level,
                "dist": dist_key,
                "file_scores": scores
            }
            out_file = "scores_datasetE%s_levelE%s_distE%s.pk" % (dataset, level, dist_key)
            with open(out_file, "w") as f:
                cPickle.dump(result, f)

Computing:  SALAMI large_scale correlation
